# Multiple Parameter Estimations
It is often desirable to run a single optimization problem many time in order to analyse the results. In copasi this is fairly easy simply by using the `repeat` scan item. In PyCoTools we can use the `runMultiplePEs` class to the same end.   

## Get Handle to Model

In [2]:
%matplotlib inline
import os,glob

for i in glob.glob('*kholodenko.cps'):
    kholodenko_model= os.path.abspath(i)
    
print kholodenko_model
print os.path.isfile(kholodenko_model)

C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko.cps
True


## Generate Synthetic Data

In [3]:
import site
site.addsitedir('C:\Users\Ciaran\Documents\PyCoTools')

import pandas,os,PyCoTools

data_report='MultipleParameterEstimationData.txt'
##Simulate some synthetic data
TC=PyCoTools.pycopi.TimeCourse(kholodenko_model,end=1000,step_size=100,
                               intervals=10,
                               report_name=data_report)

INFO:root:__init__:26:Initializing PyCoTools
INFO:root:__init__:27:Initializing logging System
INFO:root:__init__:28:logging config file at: C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\logging_config.conf


# The RunMultiplePEs Class
The RunMultiplePEs class is an interface to the ParameterEstimation class and accepts all of the same keyword parameters with a few extra:

    1. `copy_number`          --> Number of times to copy a copasi file. All of these will be run simultaneously. 
    2. `pe_number`            --> How many parameter estimations to conduct **per model**
    3. `results_directory`    --> Name of the directory to put the results (created if not exists)

The total number of estimations that can be carried out simultaneously is $copy\_number\cdot pe\_number$.  

In [3]:
import pandas,os,PyCoTools
RMPE=PyCoTools.pycopi.RunMultiplePEs(kholodenko_model,data_report,
                                     copy_number=4,
                                     pe_number=2) #--> 8 total
RMPE.write_config_template()
RMPE.set_up()
RMPE.run()

INFO:PyCoTools.pycopi:pycopi:4043:creating a directory for analysis in : 

C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\MultipleParameterEstimationResults
INFO:PyCoTools.pycopi:pycopi:3939:writing PE config template for model: C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko.cps
INFO:PyCoTools.pycopi:pycopi:924:footer is: <Element Footer at 0xb000ac8>
INFO:PyCoTools.pycopi:pycopi:3921:Copying copasi file 4 times
INFO:PyCoTools.pycopi:pycopi:3957:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko.cps
INFO:PyCoTools.pycopi:pycopi:3957:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko_1.cps
INFO:root:pycopi:3361:defining report
INFO:PyCoTools.pycopi:pycopi:3957:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko_2.cps
INFO:PyCoTools.pycopi:pycopi:924:footer is: <Element Footer at 0xbc64e48>


All models are executed using CopasiSE at the same time. For this reason, if you have too many models running at once on a single machine, the machine will slow and be unusable until the estimations have finished. My own machine (i7, 16GB RAM) can comfortably run around 6 or 7 models at once before things start to slow. If using a SunGridEngine based job scheduler you can use `run = SGE` to submit the jobs automatically. If your using a different job scheduler it is straight forward to write another function within the `pycopi.Run` class to support your scheduler. Feel free to contact me for advice.   

## Custom Results Directory
By default the parameter estimation data files are saved to a folder defined by the `results_directory` keyword. By default this is `MultipleParameterEstimationResults` in the same directory as your model. It is sometimes useful to have control over where the results go (testing slight variations in estimation settings for example). Changing the `results_directory` argument specifies a different directory to put the results in (PyCoTools creates it if it does not exist). 

In [4]:
import pandas,os,PyCoTools
results_directory = 'Fit1Results'
RMPE=PyCoTools.pycopi.RunMultiplePEs(kholodenko_model,data_report,
                                     copy_number=4,
                                     pe_number=2,
                                     results_directory = results_directory,
                                     append = False)
RMPE.write_config_template()
RMPE.set_up()
RMPE.run()

INFO:PyCoTools.pycopi:pycopi:4051:creating a directory for analysis in : 

C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Fit1Results
INFO:PyCoTools.pycopi:pycopi:3947:writing PE config template for model: C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko.cps
INFO:PyCoTools.pycopi:pycopi:3929:Copying copasi file 4 times
INFO:PyCoTools.pycopi:pycopi:3965:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko.cps
INFO:PyCoTools.pycopi:pycopi:3965:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko_1.cps
INFO:root:pycopi:3360:defining report
INFO:PyCoTools.pycopi:pycopi:3965:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko_2.cps
INFO:PyCoTools.pycopi:pycopi:3965:setting up scan for model : C:\Users\Ciaran\Documents\PyCoTools\PyCoTools\PyCoToolsTutorial\Kholodenko_3.cps
INFO:root:pycopi:3360:defini

Copasi does not offer a way of giving parameter estimation data appropriate headers. outputing column headings so we have to use a workaround. To get parameter estimation data files with meaningful headers, wait until all the Copasi simulation engines are finished running and then use the `format_output()` method of the RunMultiplePEs class. To monitor progress, look either at the task manager or for when the size of the output files changes from 0kb. 

In [5]:
# print pandas.read_csv(RMPE['results_directory'])
print PyCoTools.PEAnalysis.ParsePEData(RMPE['results_directory'])

IOError: Initializing from file failed

In [ ]:
RMPE.format_output()

We can now parse the data in the results folder into python directly using the PEAnalysis module. 

In [ ]:
print PyCoTools.PEAnalysis.ParsePEData(RMPE.kwargs['results_directory']).data.transpose()

Notice that we tried to run 8 parameter estimations and we only have 4 in the output. This is because we ran (deliberately) the estimations with the `append = False` option. Parameter sets are overwritten when `append=False` so this option needs to be set to `True`. By default the append argument for RunMultiplePEs is set to True to help avert this problem.  

In [ ]:
import pandas,os,PyCoTools
results_directory = 'Fit2Results'
RMPE=PyCoTools.pycopi.RunMultiplePEs(kholodenko_model,data_report,
                                     copy_number=4,
                                     pe_number=2,
                                     results_directory = results_directory,
                                     append = True)
RMPE.write_config_template()
RMPE.set_up()
RMPE.run()

In [ ]:
RMPE.format_output()

## visualize the data using boxplots
PyCoTools.PEAnalysis.PlotBoxplot(RMPE.kwargs['results_directory'])